In [17]:
from sportsreference.mlb.teams import Roster

giants = Roster('SFG', year = '2010', slim=True)
giants.players

{'poseybu01': 'Buster Posey',
 'huffau01': 'Aubrey Huff',
 'sanchfr01': 'Freddy Sanchez',
 'uribeju01': 'Juan Uribe',
 'sandopa01': 'Pablo Sandoval',
 'burrepa01': 'Pat Burrell',
 'rowanaa01': 'Aaron Rowand',
 'schiena01': 'Nate Schierholtz',
 'torrean02': 'Andres Torres',
 'renteed01': 'Edgar Renteria',
 'molinbe01': 'Bengie Molina',
 'ishiktr01': 'Travis Ishikawa',
 'whiteel03': 'Eli Whiteside',
 'guilljo01': 'Jose Guillen',
 'derosma01': 'Mark DeRosa',
 'bowkejo01': 'John Bowker',
 'downsma01': 'Matt Downs',
 'rossco01': 'Cody Ross',
 'fontemi01': 'Mike Fontenot',
 'velezeu01': 'Eugenio Velez',
 'rohliry01': 'Ryan Rohlinger',
 'burriem01': 'Emmanuel Burriss',
 'fordda03': 'Darren Ford',
 'linceti01': 'Tim Lincecum',
 'cainma01': 'Matt Cain',
 'zitoba01': 'Barry Zito',
 'sanchjo01': 'Jonathan Sanchez',
 'bumgama01': 'Madison Bumgarner',
 'welleto01': 'Todd Wellemeyer',
 'wilsobr01': 'Brian Wilson',
 'martijo07': 'Joe Martinez',
 'romose01': 'Sergio Romo',
 'motagu01': 'Guillermo Mota

In [20]:
from sportsreference.mlb.roster import Player
buster_posey = Player('poseybu01')
buster_posey.dataframe[["player_id", "name", "at_bats", "hits", "batting_average", "doubles", "triples", "home_runs", "runs_batted_in"]]

,at_bats,hits,batting_average,doubles,triples,home_runs,runs_batted_in
2009,17,2,0.118,0,0,0,0
2010,406,124,0.305,23,2,18,67
2011,162,46,0.284,5,0,4,21
2012,530,178,0.336,39,1,24,103
2013,520,153,0.294,34,1,15,72
2014,547,170,0.311,28,2,22,89
2015,557,177,0.318,28,0,19,95
2016,539,155,0.288,33,2,14,80
2017,494,158,0.320,34,0,12,67
2018,398,113,0.284,22,1,5,41


In [4]:
# pull in dependencies needed from sportsreference api
from sportsreference.mlb.teams import Teams
from sportsreference.mlb.teams import Roster

# to create a list of player ids:
# set up an empty array that we will append to
player_list = []

# 1) create a loop to go through years
for year in range(2019,2020):

# 2) create a loop to go through teams and pull out the player names and their ids
    for team in Teams(year):
        player_data = Roster(team.abbreviation, year = year, slim=True).players
        player_list.append(player_data)

In [5]:
# convert the dictionay of arrays to an array of arrays using list comprehension
info = [list(x.keys()) for x in player_list]

In [6]:
# flatten the list using itertools
import itertools

merged = list(itertools.chain(*info))

In [11]:
# create a pandas data frame that will show all of the player ids
import pandas as pd
df = pd.DataFrame(merged)
df

,0
0,chiriro01
1,gourryu01
2,altuvjo01
3,correca01
4,bregmal01
...,...
1562,garrere01
1563,greensh02
1564,reiniza01
1565,rossty01


In [30]:
# use .unique() on the df to clear out any duplicate players
unique_player_id = df[0].unique()
# new_df = pd.DataFrame(unique_player_id)
# new_df
unique_player_id

array(['chiriro01', 'gourryu01', 'altuvjo01', ..., 'garrere01',
       'reiniza01', 'rossty01'], dtype=object)

In [35]:
from sportsreference.mlb.roster import Player
player_df_list = []
for index, player_id in enumerate(unique_player_id):
    player_df = Player(player_id).dataframe[["player_id", "name", "at_bats", "hits", "batting_average", "doubles", "triples", "home_runs", "runs_batted_in"]]
    player_df_list.append(player_df)
    print(index, len(unique_player_id))

0 1410
1 1410
2 1410
3 1410
4 1410
5 1410
6 1410
7 1410
8 1410
9 1410
10 1410
11 1410
12 1410
13 1410
14 1410
15 1410
16 1410
17 1410
18 1410
19 1410
20 1410
21 1410
22 1410
23 1410
24 1410
25 1410
26 1410
27 1410
28 1410
29 1410
30 1410
31 1410
32 1410
33 1410
34 1410
35 1410
36 1410
37 1410
38 1410
39 1410
40 1410
41 1410
42 1410
43 1410
44 1410
45 1410
46 1410
47 1410
48 1410
49 1410
50 1410
51 1410
52 1410
53 1410
54 1410
55 1410
56 1410
57 1410
58 1410
59 1410
60 1410
61 1410
62 1410
63 1410
64 1410
65 1410
66 1410
67 1410
68 1410
69 1410
70 1410
71 1410
72 1410
73 1410
74 1410
75 1410
76 1410
77 1410
78 1410
79 1410
80 1410
81 1410
82 1410
83 1410
84 1410
85 1410
86 1410
87 1410
88 1410
89 1410
90 1410
91 1410
92 1410
93 1410
94 1410
95 1410
96 1410
97 1410
98 1410
99 1410
100 1410
101 1410
102 1410
103 1410
104 1410
105 1410
106 1410
107 1410
108 1410
109 1410
110 1410
111 1410
112 1410
113 1410
114 1410
115 1410
116 1410
117 1410
118 1410
119 1410
120 1410
121 1410
122 1410
123

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
player_df_list